# This notebook provides the commands needed to read in an NWB file with data from one day of one animal (Animal Bond, day 4) and to copy the relevant metadata into a DataJoint Schema designed to mimic NWB structure

## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
### 3) Install the franklabnwb package. This can currently be downloaded from 
     https://bitbucket.org/franklab/franklabnwb/src/master/
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/s/j4mr7riyhkar7gp/bon04.nwb?dl=0

#### Make sure we're in the franklabnwb/franklabnwb directory. 
#### Eventually this will be unnecessary because the package will properly installed

In [2]:
cd ../schema

/Users/loren/Src/NWB/nwb_datajoint/schema


#### Load all of the relevant modules

In [6]:
%load_ext autoreload
%autoreload 2


import pynwb

import os

#DataJoint and DataJoint schema
import datajoint as dj
import nwb_dj
import common_session
import common_ephys
import common_device
import common_lab
import common_task
import common_behav
import common_interval
import common_region

import createprobes


#franklabnwb functions
from franklabnwb import fl_ns_path
import franklabnwb.nspike_helpers as ns
import franklabnwb.query_helpers as qu
import franklabnwb.fl_extension as fle
import franklabnwb.fl_extension as flh

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Populate the probe schema and check that it worked. You should see entries in the probe table corresponding to different probe types

In [8]:

createprobes.add_probes()
common_device.Probe()

testing


probe_type,probe_description description of this probe,num_shanks number of shanks on this device,contact_side_numbering electrode numbers from contact side of the device
32c-2s8mm6cm-20um-40um-dl,32 channel polymide probe,2,1
tetrode,tetrode made from 4 12.5 um nichrome wires,1,1


#### Next we specify the NWB file name and populate the rest of the schema
#### Replace the code below with the text for the path to the NWB file

In [13]:
conn = dj.conn()
conn.commit_transaction()

In [9]:
nwb_file_name = '/data/mkarlsso/Bon/NWB/bon04.nwb'

In [19]:
nwb_dj.NWBPopulate([nwb_file_name])

[autoreload of common_ephys failed: Traceback (most recent call last):
  File "/Users/loren/anaconda3/envs/lfdatamigration/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/loren/anaconda3/envs/lfdatamigration/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/loren/anaconda3/envs/lfdatamigration/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/anaconda3/envs/lfdatamigration/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/loren/anaconda3/envs/lfdatamigration/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/anaconda3/envs/lfdatamigration/lib/python

file names: ['/data/mkarlsso/Bon/NWB/bon04.nwb']
Starting NWB file /data/mkarlsso/Bon/NWB/bon04.nwb
Skipping task Sleep; already in schema

Skipping task W-Alternation; already in schema

Skipping apparatus Sleep Box; already in schema

Skipping apparatus TrackA; already in schema

Skipping apparatus TrackB; already in schema



#### The code below creates a diagram of the various schema and their relationships

In [29]:
common_session.Session()

nwb_file_name,subject_id,institution_name,lab_name,session_id,session_description,session_start_time,timestamps_reference_time,experiment_description
/data/mkarlsso/Bon/NWB/bon04.nwb,Bond,UCSF,Frank Laboratory,Bon0004,data acquisition session,2006-01-01 12:00:00,2006-01-01 12:00:00,Tetrode recordings from rat during rest and task performance


In [31]:
dj.Diagram(common_session)

FileNotFoundError: [Errno 2] "dot" not found in path.

In [21]:
dj.Diagram(common_ephys)+dj.Diagram(common_session)+dj.Diagram(common_lab)+dj.Diagram(common_interval)+dj.Diagram(common_behav)

FileNotFoundError: [Errno 2] "dot" not found in path.

In [35]:
#### And now that we have the data in DataJoint, we can do queries to find time intervals we might be interested in
#### In the example below, we get all of the intervals where the task_name was sleep.

In [25]:
sleep_intervals = (common_interval.IntervalList() & (common_task.TaskEpoch() & 'task_name = "Sleep"')).fetch()
sleep_intervals

array([('/data/mkarlsso/Bon/NWB/bon04.nwb', 'task epoch 0', array([1.13614661e+09, 1.13614794e+09])),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', 'task epoch 2', array([1.13614908e+09, 1.13615003e+09])),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', 'task epoch 4', array([1.13615118e+09, 1.13615241e+09])),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', 'task epoch 6', array([1.13615411e+09, 1.13615495e+09]))],
      dtype=[('nwb_file_name', 'O'), ('interval_name', 'O'), ('valid_times', 'O')])

In [26]:
sleep_intervals['valid_times']

array([array([1.13614661e+09, 1.13614794e+09]),
       array([1.13614908e+09, 1.13615003e+09]),
       array([1.13615118e+09, 1.13615241e+09]),
       array([1.13615411e+09, 1.13615495e+09])], dtype=object)

In [28]:
common_ephys.ElectrodeConfig.ElectrodeGroup().fetch()

array([('/data/mkarlsso/Bon/NWB/bon04.nwb', '01', 2, 'NSpike acquisition system', 'tetrode 1 located in CA3 on day 4', 'Right'),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', '02', 2, 'NSpike acquisition system', 'tetrode 2 located in CA3 on day 4', 'Right'),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', '03', 3, 'NSpike acquisition system', 'tetrode 3 located in CA1 on day 4', 'Right'),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', '04', 3, 'NSpike acquisition system', 'tetrode 4 located in CA1 on day 4', 'Right'),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', '05', 3, 'NSpike acquisition system', 'tetrode 5 located in CA1 on day 4', 'Right'),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', '06', 4, 'NSpike acquisition system', 'tetrode 6 located in ? on day 4', 'Right'),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', '07', 2, 'NSpike acquisition system', 'tetrode 7 located in CA3 on day 4', 'Right'),
       ('/data/mkarlsso/Bon/NWB/bon04.nwb', '08', 2, 'NSpike acquisition system', 'tetrode 8 locate